### Deep learning

上节课从Logistic regression（LR）的Limitation中引出Neural Network。多个Neuron的cascading就是neural network.而在这个neural network中同时学习LR中参数的方法被称为Deep learning（Learning 就是学习的意思）.

本节继续围绕机器学习的三大部分。Step1: define a set of function, Step2: goodness of a function, Step3: pick the best function这三个步骤来展开讨论。

Deep learning不是从一开始就有的。它经历了多次发展，其发展轨迹如下。

|   Time  |                     Ups and Downs                    |                       Notes                       |
|:-------:|:----------------------------------------------------:|:-------------------------------------------------:|
|   1958  | Perceptron                                           | (linear model)                                    |
|   1969  | Perceptron has limitation                            |                                                   |
|  1980s  | Multi-layer perceptron                               | Do not have significant difference from DNN today |
|   1986  | Backpropagation                                      | Usually more than 3 hidden layers is not helpful  |
|   1989  | 1 hidden layer is “good enough”, why deep?           |                                                   |
|   2006  | RBM initialization                                   |                                                   |
|   2009  | GPU                                                  |                                                   |
|   2011  | Start to be popular in speech recognition            |                                                   |
|   2012  | win ILSVRC image competition                         |                                                   |
|  2015.2 | Image recognition surpassing human-level performance |                                                   |
|  2016.3 | Alpha GO beats Lee Sedol                             |                                                   |
| 2016.10 | Speech recognition system as good as humans          |                                                   |

可以看出，1980年代就已经出现了DNN的原型，1986年才提出了BP过程用于更新参数；2009年才发现，通过使用GPU进行加速可以极大缩短训练时间；deep 
learning最开始兴起于语音识别，随后才是CV和NLP.

#### Step1: Neural Network

首先注意Neuron以及由Neuron构成的Neural Network的结构。如图

![](./ml_png/4_neural_network.png)

其中的neuron是一个LR结构。由neuron的不同的连接方式可构成不同的网络结构，所指的network parameter $\theta$是LR中的$w,b$。如果各个neuron用全连接方式进行connect，就能得到最简单的fully connect feedforward network(全连接前馈网络，FCFN)，该network如下图。

![](./ml_png/4_fc_network.png)

该网络的输入是一个vector，输出也是一个vector，整个网络构成了一个映射函数$f$。比如输入变量是$\left[ \begin{array} -1 \\ 1 \end{array} \right]$，那么输出vector为$f \left( \left[ \begin{array} -1 \\ 1 \end{array} \right] \right) = \left[ \begin{array}00.62 \\ 0.83 \end{array} \right]$,或者输入$\left[ \begin{array} 00 \\ 0 \end{array} \right]$,output vector为$f \left( \left[ \begin{array} 00 \\ 0 \end{array} \right] \right) = \left[ \begin{array}00.51 \\ 0.85 \end{array} \right]$.

如果需要添加更多的neuron，并且给每一组neuron定义一个名称，以方便后续讨论。那么FCFN就包含了Input, Layer1, Layer2, ... ,LayerL, Output.其中Input所在的层是input layer, Layer1, Layer2, ... ,LayerL就称为Hidden Layers，Output所在的层就是Output Layer,如图。

![](./ml_png/4_FCFN.png)

当neural network中hidden layer堆叠了很多层时，我们就说它是一个deep的网络。具体堆叠了几层才算多，一般这个"多"应不少于3.下面是几个经典的deep neural network。

|      Structure     | Date | Layer | Accuracy |
|:------------------:|:----:|:-----:|:--------:|
|       AlexNet      | 2012 |   8   |   16.4%  |
|         vgg        | 2014 |   19  |   7.3%   |
| GoogleNet | 2014 |   22  |   6.7%   |
|       ResNet       | 2015 |  152  |   3.57%  |

可发现，随着网络结构的加深，top-5分类精度不断提升。


将上述deep neural network(DNN)用于实际分类时，neuron的个数并非数个，通常是成百上千个。由于GPU尤其擅长矩阵中的并行运算，因此，为加速每个neuron中的$z or a$的求值，通常使用matrix operation来展开运算。假设input layer只有两个值$\left[ \begin{array}{rr} x_1 \\ x_2 \end{array} \right]=\left[ \begin{array}{array}1 \\ -2 \end{array} \right]$，layer1只有两个neuron，那么$x_1$与layer1之间的weight为$\left[ \begin{array} 11, & -2 \end{array} \right]$,$x_2$与layer1之间的weight为$\left[ \begin{array}{rr}-1, & 1\end{array} \right]$.两个weight pairs构成一个matrix，即$W=\left[ \begin{array}{rr} 1 & -2 \\ -1 & 1\end{array} \right]$，加上biases值$\left[ \begin{array}{rr} b_1 \\ b_2 \end{array} \right]=\left[ \begin{array}{rr}1 \\ 0 \end{array} \right]$,完整的表达式为

$$
\sigma \left( Wx+b \right) =
\sigma\left( 
\left[ \begin{array}{rr} 1 & -2 \\ -1 & 1\end{array} \right] 
\left[ \begin{array}{rr}1 \\ -1 \end{array} \right] + 
\left[ \begin{array}{rr}1 \\ 0  \end{array} \right] \right) = \left[ \begin{array}{rr}0.98 \\ 0.12  \end{array} \right]
$$

将各层的输出在neural network中标示出来，并使用matrix operation的方式进行计算，有

![](./ml_png/4_nn_matrix_operation_in_figure.png)

将ouput与input直接通过一个函数连接起来，相当于将上一层Layer1的输出$a^1$作为Layer2的输入，以此类推，则有

$$\begin{align}
y&=f(x) = 
\sigma \left(  W^L · \dots · \sigma \left(   W^3  \sigma \left(   W^2  \sigma \left( W^1 x + b^1  \right)  +b^2 \right) +b^3 \right) · \dots · + b^L \right)
\end{align}$$

在输出层中，如果进行的是多分类任务，使用的依然是softmax函数。此时，中间隐藏层的作用可视为是feature extractor（自动学习）.最终网络的作用是勾勒目标对象的特征，然后对特征进行分类。下面以一个简答例子——手写数字识别来讲解。

输入是一个16 $\times $16的图像数据，首先将这个图像进行拉伸，得到一个256维的input vector.随后经过output为softmax的NN（上图）, 输出一个10维的向量（因为只有0-9共10个数字）。整个步骤可用下图展示。

![](./ml_png/4_handwriting_example.png)

自然而然地，关于deep neural network会有这样的问题。

1. 需要多少层layer？即Layer L中的L应该取值多少才算合理。对每一层Layer来说，需要多少个Neuron才算合理。

2. 既然Layer和neuron都需要定义，那么能否直接让网络自己去学习这些待定义的参数？

3. 能否直接自己设计网络结构。

第一个，layer和neuron的取值通过不断试错（trial+error）、直觉（intuition）来决定。也就是，好不好，试过才知道。（这也是DNN备受诟病的地方）第二个问题，可行，其中进化型的人工神经网络就是这样的一个方法。第三个，可自己设计一个相对合理的NN, 也就是后面要介绍的CNN(Convolutional Neural Network).



#### Step2: Goodness of function

假设$p,q$符合Bernoulli distribution,那么它们之间的交叉熵（cross entropy）为：

$$
H(\mathrm{p},\mathrm{q})=  -\sum_x \mathrm{p}(x) \ln\left( \mathrm{q}(x) \right)
$$

进一步地，往neural network中输入一张图像，经过hidden layer, output layer(softmax)得到输出值$y^n$,使用cross entropy来度量$y^n、\hat{y}^n$之间的距离，有
$$
l(y, \hat{y}) = -\sum_{i=1}^{10}{\hat{y}^i \ln y^i}
$$

![](./ml_png/4_loss_for_an_example3.png)

如果同时输入$N$张图片，那么有Total loss:

$$
L = \sum_{n=1}^{N}{l^n}
$$

![](./ml_png/4_total_loss.png)


#### Step3: Pick the best function (Gradient Descent)

与先前的讨论一样，先计算Loss相对于每个weight的偏微分。再按照gradient descent的方向进行参数的更新。该过程可描述为：

![](./ml_png/4_gradient_descent.png)

参数weight,biases的不断更新就是learning的过程。在deep网络上进行所有参数的learning就是deep learning的主要任务。其中backpropagation就是实现快速计算gradient的一个过程。该过程可通过多种深度学习框架来快速实现。如Tensorflow/pytorch等。